## Problem 1: Generating Random Boolean Functions

### Problem Description

The [Deutsch–Jozsa algorithm](https://quantum.cloud.ibm.com/learning/en/modules/computer-science/deutsch-jozsa) works with functions that accept a fixed number of Boolean inputs and return a single Boolean output.
As described on [IBM Quantum Learning](https://quantum.cloud.ibm.com/learning/en/courses/fundamentals-of-quantum-algorithms/quantum-query-algorithms/deutsch-jozsa-algorithm), each function is guaranteed to be either **constant** (always returns `False` or always returns `True`) or **balanced** (returns `True` for exactly half of the possible input combinations).

The task is to write a Python function `random_constant_balanced` that returns a randomly chosen function from the set of constant or balanced functions taking **four** Boolean arguments as inputs.

### My Understanding

We need to build a function that, each time it is called, produces and returns a new **callable function** `f(b1, b2, b3, b4)` → `bool`.
The returned function must behave as an oracle — a black box that a caller can evaluate on inputs but cannot inspect internally.
As explained in IBM Quantum Learning's lesson on the [query model of computation](https://quantum.cloud.ibm.com/learning/en/courses/fundamentals-of-quantum-algorithms/quantum-query-algorithms/query-model-of-computation), this oracle model is central to the Deutsch–Jozsa problem: the goal is to determine a global property of the function (constant vs. balanced) while minimising the number of queries.
Later problems will test classical and quantum strategies for this task, so our generator must produce proper callable functions rather than exposing the truth table directly.

### Theory / Background

With four Boolean inputs there are $2^4 = 16$ possible input combinations.
A Boolean function $f$ maps each combination to a single output, forming its truth table.
IBM Quantum Learning's [Deterministic operations](https://quantum.cloud.ibm.com/learning/en/courses/basics-of-quantum-information/single-systems/classical-information#deterministic-operations) section describes this setup: for a classical state set $\Sigma = \{0, 1\}$, there are $|\Sigma|^{|\Sigma|^n}$ possible functions from $n$-bit inputs to a single-bit output.

For the Deutsch–Jozsa problem, only two categories matter ([Deutsch & Jozsa, 1992](https://doi.org/10.1098/rspa.1992.0167)):

- **Constant functions**: exactly **2** (all outputs `False`, or all outputs `True`).
- **Balanced functions**: $\binom{16}{8} = 12{,}870$ (choosing which 8 of the 16 inputs map to `True`).

Together these give **12,872** valid functions — a small subset of the $2^{16} = 65{,}536$ total Boolean functions of four inputs.
As de Wolf notes in his [Quantum Computing lecture notes (arXiv:1907.09415, §3.2)](https://arxiv.org/abs/1907.09415), the promise that $f$ is either constant or balanced is essential — without it, the problem becomes much harder.

### Approach

Following the truth-table-with-closure pattern from the module's [random-functions notebook](https://github.com), where the lecturer builds random Boolean functions using tuples and inner functions:

1. **Randomly choose** between constant and balanced (50/50 via [`random.choice`](https://docs.python.org/3/library/random.html#random.choice)).
2. **Build a truth table tuple** of length 16 — all same value for constant, or half `0`s and half `1`s ([shuffled](https://docs.python.org/3/library/random.html#random.shuffle)) for balanced.
3. **Wrap it in a closure** — an inner function that captures the truth table from the enclosing scope, converts four Boolean arguments to an integer index, and looks up the result. This technique is described in the [Python documentation on closures](https://docs.python.org/3/reference/executionmodel.html#resolution-of-names).
4. Return the closure as a callable function.

### Discussion / Interpretation

The key design choice is returning a **callable function** rather than a raw truth table array.
This is essential because Problem 2 expects a function `determine_constant_balanced` that takes `f` as a parameter and calls it — so `f` must be callable.
This mirrors the approach used in [IBM Quantum Learning's Deutsch–Jozsa tutorial](https://quantum.cloud.ibm.com/learning/en/modules/computer-science/deutsch-jozsa), where the oracle is generated as a black-box circuit that can be queried but not inspected.

Attaching `func_type` metadata to the returned function lets us verify correctness in tests without breaking the black-box contract.
Using [`random.shuffle`](https://docs.python.org/3/library/random.html#random.shuffle) to arrange the balanced outputs guarantees exactly 8 ones and 8 zeros.
As Aaronson explains in his quantum computing lectures ([Lecture 17: Quantum Query Complexity](https://www.scottaaronson.com/qclec/17.pdf)), the classical worst case requires $2^{n-1} + 1 = 9$ queries for $n = 4$ inputs — motivating the quantum approach explored in later problems.

### Implementation

In [1]:
# Random selections.
# See: https://docs.python.org/3/library/random.html
import random

# Permutations and combinations.
# See: https://docs.python.org/3/library/itertools.html
import itertools

We first define a helper function that converts an arbitrary number of Boolean (or binary) arguments into an integer.
This lets us use the arguments as an index into a truth table tuple.
For example, `(True, False, True, True)` corresponds to the binary string `1011`, which is the integer 11.

This helper follows the same pattern used in the module's random-functions notebook, using Python's built-in [`int`](https://docs.python.org/3/library/functions.html#int) function with base 2 and the [variadic `*args` syntax](https://docs.python.org/3/tutorial/controlflow.html#arbitrary-argument-lists).

In [2]:
def bin_args_to_int(*args):
    """Convert an arbitrary number of Boolean/binary arguments to an integer.

    Each argument is treated as a binary digit (truthy = 1, falsy = 0).
    The first argument is the most significant bit.

    Returns
    -------
    int
        The integer represented by the binary arguments.
    """
    # Convert each argument to '1' or '0'.
    bits = ''.join('1' if b else '0' for b in args)
    # Parse the binary string as an integer.
    return int(bits, 2)

In [3]:
# Quick demonstration of the helper.
print(f"bin_args_to_int(False, False, False, False) = {bin_args_to_int(False, False, False, False)}")
print(f"bin_args_to_int(True,  False, True,  True)  = {bin_args_to_int(True, False, True, True)}")
print(f"bin_args_to_int(True,  True,  True,  True)  = {bin_args_to_int(True, True, True, True)}")

bin_args_to_int(False, False, False, False) = 0
bin_args_to_int(True,  False, True,  True)  = 11
bin_args_to_int(True,  True,  True,  True)  = 15


Now we define `random_constant_balanced`.
It generates a truth table tuple and returns a closure that looks up values in that tuple.

In [4]:
def random_constant_balanced():
    """Return a random constant or balanced Boolean function of four inputs.

    The returned function accepts four Boolean arguments and returns a
    single Boolean value.  It is guaranteed to be either constant (same
    output for every input) or balanced (True for exactly 8 of the 16
    possible inputs).

    Returns
    -------
    function
        A callable f(b1, b2, b3, b4) -> bool.
    """
    # Number of Boolean inputs.
    n = 4

    # Total number of input combinations: 2^4 = 16.
    table_size = 2 ** n

    # Randomly choose between constant and balanced.
    func_type = random.choice(["constant", "balanced"])

    if func_type == "constant":
        # Constant: all outputs are the same value (0 or 1).
        value = random.choice([0, 1])
        # Tuple repetition: (x,) * k gives (x, x, ..., x) of length k.
        truth_table = (value,) * table_size
    else:
        # Balanced: exactly half 0s and half 1s.
        # Start with 8 zeros followed by 8 ones as a list.
        tt_list = [0] * (table_size // 2) + [1] * (table_size // 2)
        # Shuffle in place so the 1s are randomly distributed.
        # See: https://docs.python.org/3/library/random.html#random.shuffle
        random.shuffle(tt_list)
        # Convert to a tuple (immutable).
        truth_table = tuple(tt_list)

    # Define the closure that looks up the truth table.
    def f(b1, b2, b3, b4):
        """Evaluate the Boolean function for the given four inputs."""
        return bool(truth_table[bin_args_to_int(b1, b2, b3, b4)])

    # Attach metadata for testing and verification.
    f.func_type = func_type
    f.truth_table = truth_table

    return f

### Demonstration

Let us generate a random function and inspect its behaviour.

In [5]:
# Generate a random function.
f = random_constant_balanced()

# Display its type (metadata attached for verification).
print(f"Function type: {f.func_type}")

Function type: balanced


In [6]:
# Call f on a few example inputs.
print(f"f(False, False, False, False) = {f(False, False, False, False)}")
print(f"f(True,  False, True,  False) = {f(True, False, True, False)}")
print(f"f(True,  True,  True,  True)  = {f(True, True, True, True)}")

f(False, False, False, False) = True
f(True,  False, True,  False) = False
f(True,  True,  True,  True)  = True


In [7]:
# Display the full truth table.
# itertools.product generates all input combinations.
# See: https://docs.python.org/3/library/itertools.html#itertools.product
all_inputs = list(itertools.product([False, True], repeat=4))

print("  b1  b2  b3  b4  |  f(b1,b2,b3,b4)")
print("-" * 38)
for inp in all_inputs:
    # Format each Boolean as 0/1 for readability.
    bits = "   ".join(str(int(b)) for b in inp)
    output = f(*inp)
    print(f"   {bits}  |  {output}")

  b1  b2  b3  b4  |  f(b1,b2,b3,b4)
--------------------------------------
   0   0   0   0  |  True
   0   0   0   1  |  True
   0   0   1   0  |  False
   0   0   1   1  |  False
   0   1   0   0  |  False
   0   1   0   1  |  False
   0   1   1   0  |  False
   0   1   1   1  |  True
   1   0   0   0  |  False
   1   0   0   1  |  True
   1   0   1   0  |  False
   1   0   1   1  |  False
   1   1   0   0  |  True
   1   1   0   1  |  True
   1   1   1   0  |  True
   1   1   1   1  |  True


### Testing

We verify correctness by generating many functions and checking that:

1. The returned object is callable and produces Boolean outputs.
2. **Constant** functions return the same value for all 16 inputs.
3. **Balanced** functions return `True` for exactly 8 of the 16 inputs.

In [8]:
def test_random_constant_balanced(num_trials=1000):
    """Test that generated functions are valid constant or balanced functions.

    Parameters
    ----------
    num_trials : int
        Number of random functions to generate and verify.
    """
    # All 16 possible 4-bit Boolean input combinations.
    all_inputs = list(itertools.product([False, True], repeat=4))

    constant_count = 0
    balanced_count = 0

    for _ in range(num_trials):
        func = random_constant_balanced()

        # The returned object must be callable.
        assert callable(func), "Returned object is not callable."

        # Evaluate on all 16 inputs.
        outputs = [func(*inp) for inp in all_inputs]

        # Every output must be a Boolean.
        assert all(isinstance(o, bool) for o in outputs), "Outputs are not all bool."

        # Count True values.
        true_count = sum(outputs)

        if func.func_type == "constant":
            # All outputs must be the same.
            assert true_count == 0 or true_count == 16, (
                f"Constant function has {true_count} True values (expected 0 or 16)."
            )
            constant_count += 1
        elif func.func_type == "balanced":
            # Exactly half must be True.
            assert true_count == 8, (
                f"Balanced function has {true_count} True values (expected 8)."
            )
            balanced_count += 1
        else:
            raise ValueError(f"Unknown function type: {func.func_type}")

    print(f"All {num_trials} trials passed.")
    print(f"  Constant functions generated: {constant_count}")
    print(f"  Balanced functions generated: {balanced_count}")

In [9]:
# Run the main test.
test_random_constant_balanced()

All 1000 trials passed.
  Constant functions generated: 524
  Balanced functions generated: 476


In [10]:
# Additional edge-case tests.

# Test 1: Truth table must have exactly 16 entries.
func = random_constant_balanced()
assert len(func.truth_table) == 16, "Truth table should have 16 entries."
print("Test 1 passed: truth table has 16 entries.")

# Test 2: Separate calls return independent function objects.
f1 = random_constant_balanced()
f2 = random_constant_balanced()
assert f1 is not f2, "Each call should return a new function object."
print("Test 2 passed: separate calls return independent functions.")

# Test 3: Function enforces exactly four arguments.
try:
    func(True, False)
    print("Test 3 failed: should have raised an error for wrong argument count.")
except TypeError:
    print("Test 3 passed: function correctly requires exactly four arguments.")

Test 1 passed: truth table has 16 entries.
Test 2 passed: separate calls return independent functions.
Test 3 passed: function correctly requires exactly four arguments.


## Problem 2: Classical Testing for Function Type

### Problem Description

[Deutsch's algorithm](https://quantum.cloud.ibm.com/learning/en/courses/fundamentals-of-quantum-algorithms/quantum-query-algorithms/deutsch-algorithm) demonstrates a [potential advantage of quantum computing](https://www.quantamagazine.org/john-preskill-explains-quantum-supremacy-20191002/) over classical computation.
To understand this advantage, we must first understand the classical cost of the problem.

The task is to write a Python function `determine_constant_balanced` that takes as input a function `f` (as defined in Problem 1), analyses it, and returns the string `"constant"` or `"balanced"`.
We must also discuss the efficiency of our solution and determine the maximum number of calls to `f` needed for 100% certainty.

### My Understanding

We receive a black-box function `f` from Problem 1 and can only learn about it by calling it on specific inputs.
Each call counts as one **query** — and the goal is to classify `f` using as few queries as possible.
As described in IBM Quantum Learning's [query model of computation](https://quantum.cloud.ibm.com/learning/en/courses/fundamentals-of-quantum-algorithms/quantum-query-algorithms/query-model-of-computation), the number of queries is the standard measure of efficiency for oracle problems.

### Theory / Background

For $n = 4$ inputs, there are $2^n = 16$ possible input combinations.
A constant function returns the same value for all 16 inputs; a balanced function returns `True` for exactly 8 and `False` for 8.

The key insight, as explained in Aaronson's [Lecture 17](https://www.scottaaronson.com/qclec/17.pdf), is that in the **worst case** we may query a balanced function and get the same output every time by bad luck.
If we see 9 identical outputs ($2^{n-1} + 1 = 9$), we can be certain the function is constant — because a balanced function only has 8 copies of any given value.
Conversely, the moment we see **two different outputs**, we know the function is balanced.

The [Deutsch–Jozsa page on IBM Quantum Learning](https://quantum.cloud.ibm.com/learning/en/modules/computer-science/deutsch-jozsa) highlights that this classical worst case of $2^{n-1} + 1$ queries is exactly what the quantum algorithm improves upon — solving the same problem with a single query.

### Approach

We use an **early-exit** strategy:

1. Query `f` on the first input and record its output.
2. For each subsequent input, query `f` and compare to the first output.
3. If a **different** output is found → return `"balanced"` immediately.
4. If we reach $2^{n-1} + 1 = 9$ identical outputs → return `"constant"` (no balanced function can produce 9 identical values).

This avoids querying all 16 inputs in most cases.

### Discussion / Interpretation

The efficiency depends on the function type and which inputs happen to be queried first:

| Scenario | Queries needed | Explanation |
|----------|---------------|-------------|
| **Best case** | 2 | The first two inputs give different outputs → balanced |
| **Worst case** | $2^{n-1} + 1 = 9$ | All queried inputs happen to give the same output |
| **Constant function** | Always 9 | Must always check 9 inputs to rule out balanced |

As de Wolf explains in his [lecture notes (arXiv:1907.09415, §3.2)](https://arxiv.org/abs/1907.09415), a deterministic classical algorithm cannot do better than $2^{n-1} + 1$ queries in the worst case.
This is the gap that the Deutsch–Jozsa algorithm closes — it solves the same problem with **just one** quantum query, regardless of $n$.

In [11]:
def determine_constant_balanced(f):
    """Determine whether a four-input Boolean function is constant or balanced.

    Uses an early-exit strategy: compares each output to the first and
    stops as soon as certainty is reached.

    Parameters
    ----------
    f : callable
        A function taking four Boolean arguments, as returned by
        random_constant_balanced() in Problem 1.

    Returns
    -------
    str
        "constant" or "balanced".
    """
    # Number of inputs.
    n = 4

    # Generate all possible input combinations.
    # See: https://docs.python.org/3/library/itertools.html#itertools.product
    all_inputs = list(itertools.product([False, True], repeat=n))

    # Query the first input and record the result.
    first_output = f(*all_inputs[0])

    # Track number of queries made.
    queries = 1

    # Query remaining inputs, comparing each to the first.
    for inp in all_inputs[1:]:
        current_output = f(*inp)
        queries += 1

        # If we find a different output, f is definitely balanced.
        if current_output != first_output:
            return "balanced"

        # If we have seen 2^(n-1) + 1 = 9 identical outputs,
        # f must be constant (a balanced function has at most 8 of each value).
        if queries >= (2 ** (n - 1)) + 1:
            return "constant"

    # Fallback: all 16 outputs were the same.
    return "constant"

### Demonstration

We generate a random function, classify it, and compare the result against the known metadata from Problem 1.

In [12]:
# Generate a random function and classify it.
f = random_constant_balanced()

result = determine_constant_balanced(f)

print(f"Actual type:    {f.func_type}")
print(f"Classified as:  {result}")
print(f"Correct:        {result == f.func_type}")

Actual type:    constant
Classified as:  constant
Correct:        True


#### Efficiency Analysis

To observe how many queries are actually needed in practice, we wrap the function with a simple counter and run many trials.

In [13]:
def count_queries(f):
    """Wrapper that counts how many times f is called.

    Returns
    -------
    tuple
        (result_string, query_count)
    """
    # Simple mutable counter using a list.
    counter = [0]

    def tracked(*args):
        """Increment counter then call original function."""
        counter[0] += 1
        return f(*args)

    # Copy metadata so determine_constant_balanced sees a normal function.
    tracked.func_type = f.func_type

    result = determine_constant_balanced(tracked)
    return result, counter[0]

In [14]:
# Run 10 trials and display results.
print(f"{'Actual':<12} {'Predicted':<12} {'Queries':<10} {'Status'}")
print("-" * 46)

for _ in range(10):
    func = random_constant_balanced()
    prediction, num_queries = count_queries(func)
    status = "PASS" if prediction == func.func_type else "FAIL"
    print(f"{func.func_type:<12} {prediction:<12} {num_queries:<10} {status}")

Actual       Predicted    Queries    Status
----------------------------------------------
balanced     balanced     2          PASS
constant     constant     9          PASS
balanced     balanced     4          PASS
balanced     balanced     2          PASS
constant     constant     9          PASS
balanced     balanced     3          PASS
balanced     balanced     2          PASS
balanced     balanced     2          PASS
balanced     balanced     2          PASS
constant     constant     9          PASS


### Testing

We verify correctness over 1000 trials and confirm that the query count never exceeds 9.

In [15]:
def test_determine_constant_balanced(num_trials=1000):
    """Test that determine_constant_balanced correctly classifies functions.

    Also verifies that the query count never exceeds 2^(n-1) + 1 = 9.

    Parameters
    ----------
    num_trials : int
        Number of random functions to test.
    """
    max_queries_seen = 0

    for _ in range(num_trials):
        func = random_constant_balanced()
        prediction, num_queries = count_queries(func)

        # Classification must match the known type.
        assert prediction == func.func_type, (
            f"Misclassified {func.func_type} as {prediction}."
        )

        # Query count must not exceed the theoretical maximum.
        assert num_queries <= 9, (
            f"Used {num_queries} queries (max allowed is 9)."
        )

        max_queries_seen = max(max_queries_seen, num_queries)

    print(f"All {num_trials} trials passed.")
    print(f"  Maximum queries used in any trial: {max_queries_seen}")

In [16]:
# Run the tests.
test_determine_constant_balanced()

All 1000 trials passed.
  Maximum queries used in any trial: 9


In [17]:
# Additional edge-case tests.

# Test 1: A known constant-False function.
def const_false(b1, b2, b3, b4):
    """Always returns False."""
    return False

const_false.func_type = "constant"
assert determine_constant_balanced(const_false) == "constant"
print("Test 1 passed: constant-False function correctly classified.")

# Test 2: A known constant-True function.
def const_true(b1, b2, b3, b4):
    """Always returns True."""
    return True

const_true.func_type = "constant"
assert determine_constant_balanced(const_true) == "constant"
print("Test 2 passed: constant-True function correctly classified.")

# Test 3: A known balanced function (XOR of first two inputs).
def balanced_xor(b1, b2, b3, b4):
    """Returns True when b1 XOR b2 is True — balanced by symmetry."""
    return b1 != b2

balanced_xor.func_type = "balanced"
assert determine_constant_balanced(balanced_xor) == "balanced"
print("Test 3 passed: balanced XOR function correctly classified.")

Test 1 passed: constant-False function correctly classified.
Test 2 passed: constant-True function correctly classified.
Test 3 passed: balanced XOR function correctly classified.


## Problem 3: Quantum Oracles

### Problem Description

[Deutsch's algorithm](https://quantum.cloud.ibm.com/learning/en/courses/fundamentals-of-quantum-algorithms/quantum-query-algorithms/deutsch-algorithm) is the simplest example of a quantum algorithm using superposition to determine a global property of a function with a single evaluation.
In the single-input case, there are four possible Boolean functions.
The task is to create the appropriate quantum oracles for each of these four functions using [Qiskit](https://www.ibm.com/quantum/qiskit), demonstrate their use, and explain how each oracle implements its corresponding function.

In [ ]:
# Quantum circuits.
# See: https://www.ibm.com/quantum/qiskit
import qiskit

# Quantum simulator.
# See: https://qiskit.github.io/qiskit-aer/index.html
import qiskit_aer as aer

In [ ]:
def make_oracle_f1():
    """Oracle for f1(x) = 0 (constant zero).

    f(x) = 0 for all x, so the ancilla is never flipped.
    No gates are needed — this is the identity operation.

    Returns
    -------
    qiskit.QuantumCircuit
        A 2-qubit oracle circuit.
    """
    oracle = qiskit.QuantumCircuit(2)
    oracle.barrier()
    # No gates: f(x) = 0 means y ⊕ 0 = y (ancilla unchanged).
    oracle.barrier()
    return oracle

In [ ]:
def make_oracle_f2():
    """Oracle for f2(x) = x (identity / balanced).

    f(x) = x, so the ancilla is flipped when the input is |1>.
    This is implemented with a single CX (controlled-NOT) gate.

    Returns
    -------
    qiskit.QuantumCircuit
        A 2-qubit oracle circuit.
    """
    oracle = qiskit.QuantumCircuit(2)
    oracle.barrier()
    # CX: flip ancilla (qubit 1) when input (qubit 0) is |1>.
    oracle.cx(0, 1)
    oracle.barrier()
    return oracle

In [ ]:
def make_oracle_f3():
    """Oracle for f3(x) = NOT x (negation / balanced).

    f(x) = 1 - x, so the ancilla is flipped when the input is |0>.
    Implemented as CX followed by X on the ancilla.

    Returns
    -------
    qiskit.QuantumCircuit
        A 2-qubit oracle circuit.
    """
    oracle = qiskit.QuantumCircuit(2)
    oracle.barrier()
    # CX: flip ancilla when input is |1>.
    oracle.cx(0, 1)
    # X: flip ancilla unconditionally.
    # Combined effect: ancilla flipped when input is |0>.
    oracle.x(1)
    oracle.barrier()
    return oracle

In [ ]:
def make_oracle_f4():
    """Oracle for f4(x) = 1 (constant one).

    f(x) = 1 for all x, so the ancilla is always flipped.
    This is implemented with a single X gate on the ancilla.

    Returns
    -------
    qiskit.QuantumCircuit
        A 2-qubit oracle circuit.
    """
    oracle = qiskit.QuantumCircuit(2)
    oracle.barrier()
    # X on ancilla: always flip it since f(x) = 1 for all x.
    oracle.x(1)
    oracle.barrier()
    return oracle